# 计算图架构示例

In [1]:
import networkx as nx
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

## 创建一个有向图

In [2]:
graph = nx.DiGraph()

In [3]:
## 定义五个函数
def function1(data, count):
    count[0] += 1
    timestamp = time.time()
    print(f"Function 1 - Run {count[0]} at {timestamp}")
    data["a"] += 1
    time.sleep(1)  # 模拟耗时操作
    return data

def function2(data, count):
    count[1] += 1
    timestamp = time.time()
    print(f"Function 2 - Run {count[1]} at {timestamp}")
    data["b"] *= 2
    time.sleep(2)  # 模拟耗时操作
    return data

def function3(data, count):
    count[2] += 1
    timestamp = time.time()
    print(f"Function 3 - Run {count[2]} at {timestamp}")
    data["c"] -= 1
    time.sleep(3)  # 模拟耗时操作
    return data

def function4(data, count):
    count[3] += 1
    timestamp = time.time()
    print(f"Function 4 - Run {count[3]} at {timestamp}")
    data["d"] = data["a"] * data["b"]
    time.sleep(1)  # 模拟耗时操作
    return data

def function5(data, count):
    count[4] += 1
    timestamp = time.time()
    print(f"Function 5 - Run {count[4]} at {timestamp}")
    data["e"] = data["a"] + data["b"] + data["c"] + data["d"]
    time.sleep(1)  # 模拟耗时操作
    return data

## 添加节点和边

In [4]:
graph.add_node(function1)
graph.add_node(function2)
graph.add_node(function3)
graph.add_node(function4)
graph.add_node(function5)
graph.add_edge(function1, function4)
graph.add_edge(function2, function5)
graph.add_edge(function3, function4)
graph.add_edge(function4, function5)

In [5]:
# 执行顺序
execution_order = list(nx.topological_sort(graph))

# 统一数据结构
data = {
    "a": 1,
    "b": 2,
    "c": 3
}

count = [0, 0, 0, 0, 0]

def execute_functions(execution_order, data, count, max_workers=8):
    def execute_function(func, data, count):
        return func(data, count)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(execute_function, node, data, count): node for node in execution_order}
        for future in as_completed(futures):
            node = futures[future]
            try:
                data = future.result()
            except Exception as e:
                print(f"Function {node.__name__} failed to execute: {e}")
    
    return data

execution_order = list(nx.topological_sort(graph))
print(f"Function 0 - Run at {time.time()}")

final_data = execute_functions(execution_order, data, count, max_workers=3)
print("Final Data:", final_data)
final_data = execute_functions(execution_order, final_data, count, max_workers=8)
print("Final Data:", final_data)

Function 0 - Run at 1699377842.6658568
Function 1 - Run 1 at 1699377842.666327
Function 2 - Run 1 at 1699377842.6665049
Function 3 - Run 1 at 1699377842.6666522
Function 4 - Run 1 at 1699377843.667532
Function 5 - Run 1 at 1699377844.6687078
Final Data: {'a': 2, 'b': 4, 'c': 2, 'd': 8, 'e': 16}
Function 1 - Run 2 at 1699377845.6718342
Function 2 - Run 2 at 1699377845.6728225
Function 3 - Run 2 at 1699377845.6733139
Function 4 - Run 2 at 1699377845.6740556
Function 5 - Run 2 at 1699377845.6750836
Final Data: {'a': 3, 'b': 8, 'c': 1, 'd': 24, 'e': 36}


可以看到在计算图中123并行执行，随后45并行执行，需要在module执行时模仿这个过程